In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_17_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-11 10:54:53.334646


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_17_01/m_17_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_17_01/17_1a_posner_task_2023-06-13_11h17.00.878/17_1a_posner_task_2023-06-13_11h17.00.878.csv
Found '1b' in file: m_17_01/17_1b_posner_task_2023-06-13_12h10.19.707/17_1b_posner_task_2023-06-13_12h10.19.707.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
len(events) 417
len(validity) 100
# of stim events 100
[    0     0 99999]
[45141     0    10]
[48981     0    40]
[50481     0    70]
[53971     0    50]
[54071     0    51]
[55492     0    40]
[56991     0    70]
[60131     0    50]
[60231     0   

/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [248] Keeping  (2023-06-13 11:26:32.051604+00:00 - 2023-06-13 11:26:32.051604+00:00 -> 447.474 - 447.474)
  [249] Keeping  (2023-06-13 11:26:32.151604+00:00 - 2023-06-13 11:26:32.151604+00:00 -> 447.574 - 447.574)
  [250] Keeping  (2023-06-13 11:26:33.491604+00:00 - 2023-06-13 11:26:33.491604+00:00 -> 448.914 - 448.914)
  [251] Keeping  (2023-06-13 11:26:34.991604+00:00 - 2023-06-13 11:26:34.991604+00:00 -> 450.414 - 450.414)
  [252] Keeping  (2023-06-13 11:26:38.661604+00:00 - 2023-06-13 11:26:38.661604+00:00 -> 454.084 - 454.084)
  [253] Keeping  (2023-06-13 11:26:38.761604+00:00 - 2023-06-13 11:26:38.761604+00:00 -> 454.184 - 454.184)
  [254] Keeping  (2023-06-13 11:26:39.991604+00:00 - 2023-06-13 11:26:39.991604+00:00 -> 455.414 - 455.414)
  [255] Keeping  (2023-06-13 11:26:41.491604+00:00 - 2023-06-13 11:26:41.491604+00:00 -> 456.914 - 456.914)
  [256] Keeping  (2023-06-13 11:26:45.371604+00:00 - 2023-06-13 11:26:45.371604+00:00 -> 460.794 - 460.794)
  [257] Keeping  (2023-06-13

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
417 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 417 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.8s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 50.8s.


Method,fastica
Fit,80 iterations on raw data (671812 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=671812
    Range : 48981 ... 720792 =     48.981 ...   720.792 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
335 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
335 matching events found
No baseline correction applied
0 projection items activated


In [7]:
print(ica.exclude)

[1, 0]


In [8]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 2 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [9]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_17_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/cleaned/m_17_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/cleaned/m_17_01/m_17_01_pos1a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [10]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/cleaned/m_17_01
├── m_17_01_pos1a.eeg
├── m_17_01_pos1a.vhdr
└── m_17_01_pos1a.vmrk

1 directory, 3 files


In [11]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_17_01/m_17_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_17_01/17_1a_posner_task_2023-06-13_11h17.00.878/17_1a_posner_task_2023-06-13_11h17.00.878.csv
Found '1b' in file: m_17_01/17_1b_posner_task_2023-06-13_12h10.19.707/17_1b_posner_task_2023-06-13_12h10.19.707.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
len(events) 414
len(validity) 100
# of stim events 100
[    0     0 99999]
[37370     0    10]
[39671     0    40]
[41171     0    60]
[44101     0    50]
[44201     0    51]
[46171     0    40]
[47671     0    80]
[50781     0    50]
[50881     0   

/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [293] Keeping  (2023-06-13 12:20:55.184527+00:00 - 2023-06-13 12:20:55.184527+00:00 -> 515.804 - 515.804)
  [294] Keeping  (2023-06-13 12:20:55.284527+00:00 - 2023-06-13 12:20:55.284527+00:00 -> 515.904 - 515.904)
  [295] Keeping  (2023-06-13 12:20:56.514527+00:00 - 2023-06-13 12:20:56.514527+00:00 -> 517.134 - 517.134)
  [296] Keeping  (2023-06-13 12:20:58.014527+00:00 - 2023-06-13 12:20:58.014527+00:00 -> 518.634 - 518.634)
  [297] Keeping  (2023-06-13 12:21:00.924527+00:00 - 2023-06-13 12:21:00.924527+00:00 -> 521.544 - 521.544)
  [298] Keeping  (2023-06-13 12:21:01.024527+00:00 - 2023-06-13 12:21:01.024527+00:00 -> 521.644 - 521.644)
  [299] Keeping  (2023-06-13 12:21:03.024527+00:00 - 2023-06-13 12:21:03.024527+00:00 -> 523.644 - 523.644)
  [300] Keeping  (2023-06-13 12:21:04.524527+00:00 - 2023-06-13 12:21:04.524527+00:00 -> 525.144 - 525.144)
  [301] Keeping  (2023-06-13 12:21:07.694527+00:00 - 2023-06-13 12:21:07.694527+00:00 -> 528.314 - 528.314)
  [302] Keeping  (2023-06-13

In [13]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
414 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 414 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [14]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-06-13 12:12:59.051527+00:00 - 2023-06-13 12:24:04.031527+00:00
  [0] Keeping  (2023-06-13 12:12:59.051527+00:00 - 2023-06-13 12:12:59.051527+00:00 -> 39.671 - 39.671)
  [1] Keeping  (2023-06-13 12:13:00.551527+00:00 - 2023-06-13 12:13:00.551527+00:00 -> 41.171 - 41.171)
  [2] Keeping  (2023-06-13 12:13:03.481527+00:00 - 2023-06-13 12:13:03.481527+00:00 -> 44.101 - 44.101)
  [3] Keeping  (2023-06-13 12:13:03.581527+00:00 - 2023-06-13 12:13:03.581527+00:00 -> 44.201 - 44.201)
  [4] Keeping  (2023-06-13 12:13:05.551527+00:00 - 2023-06-13 12:13:05.551527+00:00 -> 46.171 - 46.171)
  [5] Keeping  (2023-06-13 12:13:07.051527+00:00 - 2023-06-13 12:13:07.051527+00:00 -> 47.671 - 47.671)
  [6] Keeping  (2023-06-13 12:13:10.161527+00:00 - 2023-06-13 12:13:10.161527+00:00 -> 50.781 - 50.781)
  [7] Keeping  (2023-06-13 12:13:10.261527+00:00 - 2023-06-13 12:13

Method,fastica
Fit,58 iterations on raw data (664980 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=664980
    Range : 39671 ... 704650 =     39.671 ...   704.650 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
332 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
332 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
332 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
332 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
332 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
332 matching events found
No baseline correction applied
0 projection items ac

In [16]:
ica_b.exclude

[0, 1, 7]

In [17]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 3 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [18]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/cleaned/m_17_01/m_17_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [19]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/cleaned/m_17_01
├── m_17_01_pos1a.eeg
├── m_17_01_pos1a.vhdr
├── m_17_01_pos1a.vmrk
├── m_17_01_pos1b.eeg
├── m_17_01_pos1b.vhdr
└── m_17_01_pos1b.vmrk

1 directory, 6 files


# ...

In [20]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b